<a href="https://colab.research.google.com/github/sgun0315/Sysco-Mini-Project-Users-Micro-Service/blob/master/Ask_Ernie_Big_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-api-core
%pip install --upgrade google-cloud-bigquery

In [1]:
from google.colab import auth
auth.authenticate_user()

In [1]:
import vertexai

project_id = "syy-dom-np-fd67"
region = "us-central1"
vertexai.init(project=project_id, location=region)

In [152]:
BQ_DATASET_NAME = "scm_chatbot_datastore_dev"
BQ_KNOWLEDGE_DATASET_NAME = "chatbot_knowledge_datastore_dev"
BQ_SITE_DETAILS_TABLE_NAME = "site_region_market"
BQ_TABLE_SCHEMA_KNOWLEDGE_TABLE_NAME = "table_details"
BQ_COLUMN_SCHEMA_KNOWLEDGE_TABLE_NAME = "table_column_details"
BQ_BUSINESS_KNOWLEDGE_TABLE_NAME = "business_rules"
BQ_HISTORY_SQL_TABLE_NAME = "known_good_sqls"

APPLICATION_NAME = "Merchandising"
GOOGLE_CLOUD_QUOTA_PROJECT = "syy-dom-np-fd67"
GOOGLE_CLOUD_QUOTA_PROJECT_REGION = "us-central1"

In [214]:
import logging
from typing import Literal

from google.api_core.exceptions import GoogleAPICallError, BadRequest, ServiceUnavailable
from google.cloud import bigquery

logger = logging.getLogger(__name__)

class BQConnector:
    """Big Query Connector Implementation."""

    def __init__(self, source, project_id, region):
        self.source = source

        self.project_id = project_id
        self.region = region
        self.dataset_name = BQ_DATASET_NAME
        self.chatbot_knowledge_dataset = BQ_KNOWLEDGE_DATASET_NAME

        self._client = bigquery.Client(project=project_id, location=region)

    def execute_query(self, query):
        """Execute the given query and return the result in dataframe"""
        try:
            return self._client.query_and_wait(query).to_dataframe()
        except (TimeoutError, GoogleAPICallError, BadRequest, ServiceUnavailable) as e:
            logger.error("Error Executing Query: %s", e)
            raise ResourceConnectionError(f"Error executing query: {e}", service="BigQuery", errors=e) from e

    def retrieve_site_details(self):
        site_details_table_name = BQ_SITE_DETAILS_TABLE_NAME
        site_details_query = f"""
            SELECT site_id, site_name, region_name, market_name
            FROM `{self.project_id}.{self.dataset_name}.{site_details_table_name}`
        """

        results = self.execute_query(site_details_query)
        site_details_list = []
        for index, row in results.iterrows():
            site_details_list.append({
                "site_id": row["site_id"],
                "site_name": row["site_name"],
                "region_name": row["region_name"],
                "market_name": row["market_name"]
            })

        return site_details_list

    def retrieve_matches(
            self,
            mode: Literal["table", "column", "example", "business_rule"],
            qe,
            similarity_threshold,
            limit,
    ):
        """This function retrieves the most similar table_schema and column_schema.

        Arguments:
            mode {Literal['table', 'column', or 'example']} -- Mode for the retriever.
            qe {str} -- User Question Embedding
            similarity_threshold {int} -- Similarity Threshold to filter results
            limit {int} -- Query result set limit

        Return:
            result {pd.DataFrame} -- Result set from DB
        """
        logger.info("Searching %s. Threshold: %s. Limit: %s", mode, similarity_threshold, limit)

        embeddings_retrieval_sql = """
        (select base.content as content
        from vector_search(
                    TABLE `{}`,
                    "embedding",
                    (SELECT {} as qe),
                    top_k=> {},
                    distance_type=>"DOT_PRODUCT"
        )
        where (1 - distance > {}) and base.source_type = "{}")
        """
        search_result_txt = ""

        if mode == "table":
            table_name = BQ_TABLE_SCHEMA_KNOWLEDGE_TABLE_NAME
            search_result_txt = "Schema(values):"

        elif mode == "column":
            table_name = BQ_COLUMN_SCHEMA_KNOWLEDGE_TABLE_NAME
            search_result_txt = "Column name(type):"

        elif mode == "business_rule":
            table_name = BQ_BUSINESS_KNOWLEDGE_TABLE_NAME
            embeddings_retrieval_sql = """(
                select base.ruleset as content, base.content as identifier, (1 - distance) AS similarity
                from vector_search (
                    TABLE `{}`,
                    "embedding",
                    (select {} as qe),
                    top_k=> {},
                    distance_type=>"COSINE"
                )
                where (1 - distance > {}) and base.source_type = "{}")
            """

        elif mode == "example":
            table_name = BQ_HISTORY_SQL_TABLE_NAME
            embeddings_retrieval_sql = """(
                select base.intent as user_question, base.template_sql as generated_sql
                from vector_search ( TABLE `{}`, "embedding", (select {} as qe), top_k=> {}, distance_type=>"COSINE")
                where (1 - distance > {}) and base.source_type = "{}")
            """
        else:
            raise ValueError("No valid mode. Must be either table, column, or example")

        embeddings_retrieval_sql = embeddings_retrieval_sql.format(
            f"{self.project_id}.{self.chatbot_knowledge_dataset}.{table_name}",
            qe,
            limit,
            similarity_threshold,
            self.source,
        )
        logger.debug(embeddings_retrieval_sql)

        results = self.execute_query(embeddings_retrieval_sql)

        # CHECK RESULTS
        if len(results) == 0:
            logger.info("Did not find any results. Adjust the query parameters.")
            return search_result_txt

        if mode in {"table", "column"}:
            for _, r in results.iterrows():
                search_result_txt = search_result_txt + r["content"] + "\n"

        elif mode == "business_rule":
            for _, r in results.iterrows():
                rule = r["content"]
                score = round(r["similarity"], 3)
                search_result_txt += f"{rule} (score: {score})\n"

        elif mode == "example":
            search_result_txt = ""
            for _, r in results.iterrows():
                example_user_question = r["user_question"]
                example_sql = r["generated_sql"]
                search_result_txt = (
                        search_result_txt
                        + "\n Example_question: "
                        + example_user_question
                        + " | Example_SQL: "
                        + example_sql.replace('"', "'")
                        + "\n"
                )
        # Remove additional spaces & whitespaces
        search_result_txt = search_result_txt.strip()
        logger.debug("Mode: %s Semantic Search Result: \n%s", mode, search_result_txt)
        return search_result_txt

In [215]:
import logging
from typing import Optional, List, Literal

from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

logger = logging.getLogger(__name__)


class EmbedderAgent:
    """Embedder Agent Implementation"""

    agent_type: str = "EmbedderAgent"

    def __init__(self, embeddings_model: str = "text-embedding-004"):
        self.embeddings_model = embeddings_model
        self.model = TextEmbeddingModel.from_pretrained(model_name=embeddings_model)
        logger.info("Embedder Agent Initialized in the given project.")

    async def create(
            self,
            questions,
            task: Literal["RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT", "SEMANTIC_SIMILARITY"] = "SEMANTIC_SIMILARITY",
            dimensionality: Optional[int] = 768,
    ) -> List[List[float]]:
        """Text embedding with a Large Language Model."""

        inputs = [TextEmbeddingInput(text, task) for text in questions]
        kwargs = {"output_dimensionality": dimensionality}
        embeddings = await self.model.get_embeddings_async(inputs, **kwargs)
        return [embedding.values for embedding in embeddings]

In [223]:
VECTOR_BUSINESS_RULES_MATCH_SCORE_MATCHES = 0.6

async def get_embeddings_for_text(text):
    """Get Text Embeddings from Model"""
    embedder = EmbedderAgent(embeddings_model='text-embedding-004')
    embeddings = await embedder.create([text])
    return embeddings[0]

In [224]:
bq_connector = BQConnector(
    APPLICATION_NAME,
    GOOGLE_CLOUD_QUOTA_PROJECT,
    GOOGLE_CLOUD_QUOTA_PROJECT_REGION
)

In [225]:
async def get_business_rule(message):
  embedded_question = await get_embeddings_for_text(message)


  return bq_connector.retrieve_matches(
      "business_rule",
      embedded_question,
      VECTOR_BUSINESS_RULES_MATCH_SCORE_MATCHES,
      5
  )


In [226]:
queries = [
    ["Show me 3 late POs at site 048", "Late PO"],
    ["How many late POs at site 048", "Late PO"],
    ["Show me 10 late POs at site 048", "Late PO"],
    ["SHow me first 4 late POs at site 002", "Late PO"],
    ["Show me the top 4 late purchase orders at site 002", "Late PO"],
    ["How many late pos at site 002", "Late PO"],
    ["How many late pos at 048, 002", "Late PO"],
    ["How many late PO's do I have at site 048", "Late PO"],
    ["What's the current tally of late POs at site 048", "Late PO"],
    ["How many late PO's do I have at market south", "Late PO"],
    ["How many late PO's do I have at region carolinas", "Late PO"],
    ["How many late PO's do I have at site sysco charlotte", "Late PO"],
    ["Is the above PO a late PO?", "Late PO"],
    ["How many late POs are there in that site", "Late PO"],
    ["How many late POs do I have in that site?", "Late PO"],
    ["Give me a list of very old POs (older than 2 months) which I need to clean up at site 163", "PO With Older Than Given Month"],
    ["I'm looking for a list of aged POs (older than 2 months) that require cleanup in site 048.", "PO With Older Than Given Month"],
    ["Give me a list of very old POs (older than 2 months) which I need to clean up market south", "PO With Older Than Given Month"],
    ["Give me a list of very old POs (older than 2 months) which I need to clean up region carolinas", "PO With Older Than Given Month"],
    ["Give me a list of very old POs (older than 2 months) which I need to clean up at site Charlotte", "PO With Older Than Given Month"],
    ["How many unscheduled PO's? at site 164?", "Unscheduled PO"],
    ["Do we have any unscheduled POs in site 164, and if so, how many?", "Unscheduled PO"],
    ["How many unscheduled PO's? in market south?", "Unscheduled PO"],
    ["How many unscheduled PO's? at in region south", "Unscheduled PO"],
    ["How many unscheduled PO's? at site Raleigh", "Unscheduled PO"],
    ["How many unscheduled POs are there in site 002?", "Unscheduled PO"],
    ["How many unscheduled POs do I have in site 002?", "Unscheduled PO"],
    ["Show me POs received short of total quantity. Threshold hold of 5 cases or more at site 002", "Shorted PO"],
    ["Display PO's where the received quantity falls short of the total, with a minimum of 5 cases or more at site 048.", "Shorted PO"],
    ["Show me POs received short of total quantity. Threshold hold of 5 cases or more in south", "Shorted PO"],
    ["Show me POs received short of total quantity. Threshold hold of 5 cases or more in region carolinas", "Shorted PO"],
    ["Show me POs received short of total quantity. Threshold hold of 5 cases or more at site south", "Shorted PO"],
    ["Show me PO's received short of total qty. Threshold hold of 5 cases or more", "Shorted PO"],
    ["Show POs received short of total quantity by 5+ Cases at Site 048", "Shorted PO"],
    ["Give me a list of POs which are stuck in recommended status at site 164", "PO With RECOMMENDED Status"],
    ["Could you share the purchase orders that remain in the 'recommended' status at site 164?", "PO With RECOMMENDED Status"],
    ["Give me a list of POs which are stuck in recommended status in market south", "PO With RECOMMENDED Status"],
    ["Give me a list of POs which are stuck in recommended status in region carolinas", "PO With RECOMMENDED Status"],
    ["Give me a list of POs which are stuck in recommended status at site <site name>", "PO With RECOMMENDED Status"],
    ["How many unscheduled POs are that site?", "Unscheduled PO"],
    ["Is the above PO is stucked in recommended state?", "PO With RECOMMENDED Status"],
    ["How many POs do I have stuck in recommended state in that site?", "PO With RECOMMENDED Status"],
    ["PO's that have a scheduled appointment that is in the past at site 002", "Scheduled PO"],
    ["Are there POs with scheduled appointments that are in the past at site 002?", "Scheduled PO"],
    ["PO's that have a scheduled appointment that is in the past market south", "Scheduled PO"],
    ["PO's that have a scheduled appointment that is in the past region carolinas", "Scheduled PO"],
    ["PO's that have a scheduled appointment that is in the past at site Carlotte", "Scheduled PO"],
    ["How many POs do not have an Order control number at site 002", "PO Without OC"],
    ["What is the number of POs that haven't been assigned an OC number at site 048?", "PO Without OC"],
    ["How many POs do not have an OC# in market south", "PO Without OC"],
    ["How many POs do not have an OC# in region carolinas", "PO Without OC"],
    ["How many POs do not have an OC# at site charlotte", "PO Without OC"],
    ["How many Purchase orders do not have an OC number at site gulf coast", "PO Without OC"],
    ["How many POs do not have an OC# at site 164", "PO Without OC"],
    ["Is the PO 10973040 from site 002?", "Uncategorized"],
    ["How many POs do I have in site 164", "Uncategorized"],
]


In [227]:
business_rules = {
    "Late PO": "anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED'",
    "Unscheduled PO": "purchase order primary status should be 'OPEN' and purchase order secondary status should be 'ACCEPTED' and purchase order type should be 'REGULAR' and vendor id should not be equals to '359958' or '495149'. no need to consider about order control number or scheduled purchase order receipt timestamp",
    "Shorted PO": "purchase order primary status should be 'CLOSED' and if the question includes a threshold, minimum value, then total vendor short quantity should be greater than or equal to that value. If no such value is mentioned compare with 0. threshold is a non negative value. Do not use TOTAL_RECEIVED_CASES or any other fields in the filtering logic.",
    "PO With RECOMMENDED Status": "purchase order primary status should be 'RECOMMENDED' and purchase order secondary status should be 'RECOMMENDED' and replenishment system should be 'DPR'",
    "PO With Older Than Given Month": "(anticipated receipt date + given time interval or date) should be older than today date and purchase order primary status equals to 'OPEN' and purchase order secondary status not equals to 'PARTIALLY_RECEIVED'",
    "Scheduled PO": "date of scheduled purchase order receipt timestamp should be before today's date. purchase order primary status should be 'OPEN' and purchase order secondary status should be 'SCHEDULED'",
    "PO Without OC": "order control number is null and purchase order primary status should be 'OPEN'.",
    "Uncategorized": "Uncategorized"
}

In [228]:
async def check_query(query_with_category):
    query = query_with_category[0]
    category = query_with_category[1]
    business_rule = await get_business_rule(query)

    if business_rules[category] == business_rule:
        print("\033[92mPASSED\033[0m")
    else:
        print("\033[91mFAILED\033[0m")
        print(f"Expected: {business_rules[category]}")
        print(f"Actual: {business_rule}")
    print("\n")

In [232]:
await check_query(["Show me the top 4 late Purchase Orders at site 002", "Late PO"],)

FAILED
Expected: anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED'
Actual: (anticipated receipt date + given time interval or date) should be older than today date and purchase order primary status equals to 'OPEN' and purchase order secondary status not equals to 'PARTIALLY_RECEIVED' (score: 0.764)
anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED' (score: 0.739)
date of scheduled purchase order receipt timestamp should be before today's date. purchase order primary status should be 'OPEN' and purchase order secondary status should be 'SCHEDULED' (score: 0.698)
purchase order primary status should be 'CLOSED' and if the question includes a threshold, minimum value

In [230]:
for query_with_category in queries:
    await check_query(query_with_category)


FAILED
Expected: anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED'
Actual: anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED' (score: 0.711)
(anticipated receipt date + given time interval or date) should be older than today date and purchase order primary status equals to 'OPEN' and purchase order secondary status not equals to 'PARTIALLY_RECEIVED' (score: 0.667)
date of scheduled purchase order receipt timestamp should be before today's date. purchase order primary status should be 'OPEN' and purchase order secondary status should be 'SCHEDULED' (score: 0.658)
purchase order primary status should be 'CLOSED' and if {threshold} is given in the question, then total

CancelledError: 